In [ ]:
a = [1,2,4]
for i in a[2:end]

In [47]:
function contract(a::Array{Float64, 3},b::Array{Float64, 3},ra:: Integer,rb :: Integer,ia :: Vector{Int64},ib:: Vector{Int64})
    indices_a = Set([i for i in 1:ra]);
    indices_b = Set([i for i in 1:rb]);
    rem_a = collect(indices_a - Set(ia)); 
    rem_b = collect(indices_b - Set(ib));
    idx_a = append!(rem_a,ia);
    idx_b = append!(rem_b,ib);
    
    da1 = rem_a[1]
    for i in rem_a[2:end]
        da1*=i
    end

    da2 = idx_a[1]
    for i in idx_a[2:end]
        da2*=i
    end

    db1 = rem_b[1]
    for i in rem_b[2:end]
        db1*=i
    end

    db2 = idx_b[1]
    for i in idx_b[2:end]
        db2*=i
    end
    a1 = permutedims(a,[i for i in idx_a]);
    a1 = reshape(a1,(da1,da2));
    b1 = permutedims(b,[i for i in idx_b]);
    b1 = reshape(b1,(db1,db2));

    ba = b1*a1;
    f_index = append!(rem_a,rem_b);
    BC = reshape(ba,[idx for idx in f_index]);
    return BC
    
end 

contract (generic function with 2 methods)

In [23]:
# bond dimensions
Da = 10; # alpha()
Db = 12; # beta()
Dc = 14; # gamma()
Dd = 17; # delta
Dm = 20; # mu

A = rand(Dc,Dd);   # tensor A[gamma,delta]
B = rand(Da,Dm,Dc); # tensor B[alpha,mu,gamma]
C = rand(Db,Dm,Dd); # tensor C[beta,mu,delta]

In [49]:
contract!(a=B,b=C,ra=3,rb=3,ia=[2],ib=[2])

MethodError: MethodError: no method matching contract!(; a::Array{Float64, 3}, b::Array{Float64, 3}, ra::Int64, rb::Int64, ia::Vector{Int64}, ib::Vector{Int64})

Closest candidates are:
  contract!(!Matched::Any, !Matched::Any, !Matched::Integer, !Matched::Integer, !Matched::Vector{Int64}, !Matched::Vector{Int64}) got unsupported keyword arguments "a", "b", "ra", "rb", "ia", "ib"
   @ Main c:\Users\navan\Desktop\Tensor Network 22\TN_Tutorials_2022\TN_Tutorials_2022\E02TensorNetworkBasics_jl\j_T02.1_TensorContraction\answer.ipynb:1


### Answer Given

In [63]:
function contract(A:: Array{Float64},rankA::Integer,idx_A::Vector{Int64},B:: Array{Float64},rankB::Integer,idx_B::Vector{Int64})
    #Checks
    Asz = size(A); Bsz = size(B);
    if length(Asz) != rankA
        error("ERR: Input tensor A has a different rank from input rankA.")
    end
    
    if length(Bsz) != rankB
        error("ERR: Input tensor B has a different rank from input rankB.")
    end
    
    if length(idx_A) != length(idx_B)
        error("ERR: Different # of leg indices to contract for tensors A & B.")
    end

    ## indices not to be contracted
    idx_A2 = setdiff(1:rankA,idx_A);
    idx_B2 = setdiff(1:rankB,idx_B);

    # reshape tensors into matrices with "thick" legs
    A2 = reshape(permutedims(A,tuple(cat(dims=1,idx_A2,idx_A))...),(prod(Asz[idx_A2]),prod(Asz[idx_A]))); # note: prod([]) .== 1
    B2 = reshape(permutedims(B,tuple(cat(dims=1,idx_B,idx_B2))...),(prod(Bsz[idx_B]),prod(Bsz[idx_B2])));
    C2 = A2*B2; # matrix multiplication

    # size of C
    if (length(idx_A2) + length(idx_B2)) > 1
        Cdim = (Asz[idx_A2]...,Bsz[idx_B2]...)
    else
        # place dummy legs x of singleton dimension when all the legs of A (or
        # B) are contracted with the legs of B [or A]
        Cdim = 1
    end

    # reshape matrix to tensor
    C = reshape(C2,Cdim);

    return C

    end

contract (generic function with 4 methods)

In [65]:
contract(B,C,3,3,[2],[2])

MethodError: MethodError: no method matching -(::Set{Int64}, ::Set{Int64})

In [3]:
Dset = (10:10:100); # creates a range object that represents a sequence of numbers


In [30]:
include("../../22tn-j_CodeRepository/Tensor.jl")


# pattern-1

AB = contract(A,2, [2], B, 3, [2])
ABC = contract(BC,3,[1,2])

10×14×12×17 Array{Float64, 4}:
[:, :, 1, 1] =
 6.1697   4.92149  4.84579  5.78659  …  6.35386  6.49262  6.11859  5.01938
 5.91465  5.31926  4.2948   5.1361      4.37943  5.3848   5.67709  6.18949
 5.21235  5.23623  5.00594  5.74488     4.59013  5.01693  5.00989  5.18644
 6.25958  4.45636  5.91014  6.85086     5.07993  4.05147  4.98623  5.72214
 5.7391   6.17159  5.25044  4.92714     4.45064  5.67116  5.50205  6.22807
 5.32733  5.14138  5.87897  4.62694  …  5.13125  4.88273  5.14607  5.59844
 5.14454  4.51457  5.63177  5.05111     4.34326  6.72313  5.14703  6.3349
 5.95833  5.612    5.96423  4.93057     5.20422  6.90014  4.83965  5.58331
 4.52049  4.95133  4.76529  6.08953     5.06481  3.70004  4.39938  4.64155
 5.02321  5.46792  4.27689  5.40843     6.26658  5.92983  5.76869  5.31255

[:, :, 2, 1] =
 5.52157  5.89176  5.01703  5.83928  …  5.38259  6.13656  5.78427  5.42421
 4.96429  5.79942  3.7771   4.19379     4.81223  5.25382  5.66316  5.20853
 5.06949  5.94127  4.14063  5.51092    

In [24]:
Tensor.contract(B,3,[2],C,3[2])

UndefVarError: UndefVarError: `Tensor` not defined